In [1]:
train_path = "D:\projects\ML_demo\housing\artifact\data_ingestion\2023-09-08-15-11-59\ingested_data\train\housing.csv"

In [3]:
test_path = "D:\projects\ML_demo\housing\artifact\data_ingestion\2023-09-08-15-11-59\ingested_data\test\housing.csv"

In [2]:
import pandas as pd
import numpy as np

In [3]:
import os
os.chdir(r"D:\projects\ML_demo\housing\artifact\data_ingestion\2023-09-08-15-11-59\ingested_data")
os.getcwd()

'D:\\projects\\ML_demo\\housing\\artifact\\data_ingestion\\2023-09-08-15-11-59\\ingested_data'

In [4]:
train_df=pd.read_csv("train\housing.csv")
test_df=pd.read_csv("test\housing.csv")

In [5]:
train_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,72100.0,INLAND
1,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,279600.0,NEAR OCEAN
2,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750,82700.0,INLAND
3,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,112500.0,NEAR OCEAN
4,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964,238300.0,<1H OCEAN


In [20]:
train_df.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        158
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [6]:
from housing.exception import HousingException
from housing.logger import logging
from housing.entity.config_entity import ModelTrainerConfig,TrainingPipelineConfig
from housing.util.util import read_yaml_file
from housing.constant import *
import sys,os
class Configuration:

    def __init__(self,
                 config_file_path:str=CONFIG_PATH_NAME,
                 current_time_stamp:str=CURRENT_TIME_STAMP) -> None:
        try:
            self.config_info=read_yaml_file(file_path=config_file_path)
            self.training_pipeline_config=self.get_training_pipeline_config()
            self.time_stamp=current_time_stamp
        except Exception as e:
            raise HousingException(e,sys) from e
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        try:
            logging.info("get model trainer config function started")
            artifact_dir=self.training_pipeline_config.artifact_dir
            model_trainer_config=self.config_info[MODEL_TRAINER_CONFIG_KEY]

            model_trainer_config_artifact_dir=os.path.join(artifact_dir,model_trainer_config[MODEL_TRAINER_ARTIFACT_DIR],self.time_stamp)

            trained_model_path=os.path.join(model_trainer_config_artifact_dir,
                                            model_trainer_config[MODEL_TRAINER_ARTIFACT_TRAINED_MODEL_DIR_KEY],
                                            model_trainer_config[MODEL_TRAINER_MODEL_FILE_NAME_KEY])
            
            base_accuracy=model_trainer_config[MODEL_TRAINER_BASE_ACCURACY_KEY]

            config_file_path=os.path.join(artifact_dir,
                                          model_trainer_config[MODEL_TRAINER_CONFIG_DIR_KEY],
                                          model_trainer_config[MODEL_TRAINER_CONFIG_FILENAME_KEY])
            
            Model_TrainerConfig=ModelTrainerConfig(trained_model_file_path=trained_model_path,
                                                   base_accuracy=base_accuracy,
                                                   model_config_file_path=config_file_path)
            logging.info(f"Model training config:{Model_TrainerConfig}")

            return Model_TrainerConfig
        except Exception as e:
            raise HousingException(e,sys) from e
    def get_training_pipeline_config(self) -> TrainingPipelineConfig:
        try:
            training_piepline_config=self.config_info[TRAINING_PIPELINE_CONFIG_KEY]
            artifact_dir=os.path.join(ROOT_DIR,
                                      training_piepline_config[TRAINING_PIPELINE_NAME_KEY],
                                      training_piepline_config[TRAINING_PIPELINE_ARTIFACT_DIR_KEY])
            training_piepline_config=TrainingPipelineConfig(artifact_dir=artifact_dir)
            logging.info(f"training_pipeline_config{training_piepline_config}")
            return training_piepline_config 
        except Exception as e:
            raise HousingException(e,sys) from e

In [10]:
conif=Configuration()

HousingException: 'Configuration' object has no attribute 'get_training_pipeline_config'

In [2]:
import os
os.getcwd()

'd:\\projects\\ML_demo\\Notebook'

In [5]:
os.chdir("D:\projects\ML_demo")

In [75]:
from collections import namedtuple
import importlib
GRID_SEARCH_KEY = 'grid_search'
MODULE_KEY = 'module'
CLASS_KEY = 'class'
PARAM_KEY = 'params'
MODEL_SELECTION_KEY = 'model_selection'
SEARCH_PARAM_GRID_KEY = "search_param_grid"
InitializedModelDetail = namedtuple("InitializedModelDetail",
                                    ["model_serial_number", "model", "param_grid_search", "model_name"])
GridSearchedBestModel = namedtuple("GridSearchedBestModel", ["model_serial_number",
                                                             "model",
                                                             "best_model",
                                                             "best_parameters",
                                                             "best_score",
                                                             ])
BestModel = namedtuple("BestModel", ["model_serial_number",
                                     "model",
                                     "best_model",
                                     "best_parameters",
                                     "best_score", ])

In [76]:
import yaml
class ModelFactory:
    def __init__(self, model_config_path:str = None):
        try:
            self.config : dict = ModelFactory.read_param(model_config_path)
            self.grid_serch_cv_module : str = self.config[GRID_SEARCH_KEY][MODULE_KEY]
            self.grid_serch_class_module : str = self.config[GRID_SEARCH_KEY][CLASS_KEY]
            self.grid_serch_property_data : dict= dict(self.config[GRID_SEARCH_KEY][PARAM_KEY])
            self.model_intial_config : dict = dict(self.config[MODEL_SELECTION_KEY])
            self.initlized_model_list = None
            self.grid_serched_best_model_list = None
            #print(self.get_initlized_model_list())
            print(self.config)
            print(self.model_intial_config)
        except Exception as e:
            raise HousingException(e,sys) from e
    @staticmethod
    def read_param(config_path:str)->dict:
        try:
            with open(config_path) as yaml_files:
                config:dict = yaml.safe_load(yaml_files)
                return config
        except Exception as e:
            raise HousingException(e,sys) from e
    @staticmethod
    def class_for_name(class_name:str, module_name:str):
        try:
            module = importlib.import_module(module_name)
            class_ref = getattr(module, class_name)
            return class_ref
        except Exception as e:
            raise HousingException(e,sys) from e
    @staticmethod
    def update_property_clas(insta_ref : object , property_data : dict):
        try:
            for key, value in property_data.items():
                setattr(insta_ref, key, value)
                return insta_ref
        except Exception as e:
            raise HousingException(e,sys) from e
    
    def get_initlized_model_list(self)->list[InitializedModelDetail]:
        try:
            intilized_model_list=[]
            for model_seral_number in self.model_intial_config.keys():
                model_config = self.model_intial_config[model_seral_number]
                print(model_config)
                model_object_ref = ModelFactory.class_for_name(module_name=model_config[MODULE_KEY],
                                                               class_name=model_config[CLASS_KEY])
                print(model_object_ref)
                model = model_object_ref()

                if PARAM_KEY in model_config:
                    model_object_property_data = dict(model_config[PARAM_KEY])
                    model = ModelFactory.update_property_clas(insta_ref=model, property_data = model_object_property_data)
                param_grid_serach = model_config[SEARCH_PARAM_GRID_KEY]

                model_name = f"{model_config[MODULE_KEY]}.{model_config[CLASS_KEY]}"

                model_config = InitializedModelDetail(model_serial_number=model_seral_number,
                                                      model=model,
                                                      model_name=model_name,
                                                      param_grid_search=param_grid_serach)
                intilized_model_list.append(model_config)

                self.initlized_model_list = intilized_model_list
            return self.initlized_model_list
        except Exception as e:
            raise HousingException(e,sys) from e  
    def initiate_best_parameter_search_for_initialized_models(self,initializedmodeldetails:list[InitializedModelDetail],
                                                              input_feature,
                                                              output_feature)->list[GridSearchedBestModel]:
        try:
            self.grid_serched_best_model_list=[]
            for initializedmodeldetails in initializedmodeldetails:
                grid_serched_best_model=self.initiate_best_parameter_search_for_initialized_model(
                    initized_model=initializedmodeldetails,
                    input_feature=input_feature,
                    output_feature=output_feature
                )
                self.grid_serched_best_model_list.append(grid_serched_best_model)
            return self.grid_serched_best_model_list
        except Exception as e:
            raise HousingException(e,sys) from e 
    def initiate_best_parameter_search_for_initialized_model(self,initializedmodeldetails:InitializedModelDetail,
                                                              input_feature,
                                                              output_feature)->GridSearchedBestModel:
        try:
            grid_search_cv_ref = ModelFactory.class_for_name(module_name=self.grid_serch_cv_module,
                                                             class_name=self.grid_serch_class_module)
            grid_serch_cv = grid_search_cv_ref(estimator=initializedmodeldetails.model,
                                               param_grid = initializedmodeldetails.param_grid_search)
            grid_serch_cv = ModelFactory.update_property_clas(grid_serch_cv, self.grid_serch_property_data)

            grid_serch_cv.fit(input_feature,output_feature)

            grid_search_best_model = GridSearchedBestModel(model_serial_number=initializedmodeldetails.model_serial_number,
                                                           model=initializedmodeldetails.model,
                                                           best_model=grid_serch_cv.best_estimator_,
                                                             best_parameters=grid_serch_cv.best_params_,
                                                             best_score=grid_serch_cv.best_score_)
            return grid_search_best_model
        except Exception as e:
            raise HousingException(e,sys) from e 
 

In [77]:
model1=ModelFactory("D:\projects\ML_demo\config\model.yaml")


{'grid_search': {'class': 'GridSearchCV', 'module': 'sklearn.model_selection', 'params': {'cv': 5, 'verbose': 2}}, 'model_selection': {'module_0': {'class': 'LinearRegression', 'module': 'sklearn.linear_model', 'params': {'fit_intercept': True}, 'search_param_grid': {'fit_intercept': [True, False]}}, 'module_1': {'class': 'RandomForestRegressor', 'module': 'sklearn.ensemble', 'params': {'min_samples_leaf': 3}, 'search_param_grid': {'min_samples_leaf': [6]}}}}
{'module_0': {'class': 'LinearRegression', 'module': 'sklearn.linear_model', 'params': {'fit_intercept': True}, 'search_param_grid': {'fit_intercept': [True, False]}}, 'module_1': {'class': 'RandomForestRegressor', 'module': 'sklearn.ensemble', 'params': {'min_samples_leaf': 3}, 'search_param_grid': {'min_samples_leaf': [6]}}}


In [78]:
model1.get_initlized_model_list()

{'class': 'LinearRegression', 'module': 'sklearn.linear_model', 'params': {'fit_intercept': True}, 'search_param_grid': {'fit_intercept': [True, False]}}
<class 'sklearn.linear_model._base.LinearRegression'>
{'class': 'RandomForestRegressor', 'module': 'sklearn.ensemble', 'params': {'min_samples_leaf': 3}, 'search_param_grid': {'min_samples_leaf': [6]}}
<class 'sklearn.ensemble._forest.RandomForestRegressor'>


[InitializedModelDetail(model_serial_number='module_0', model=LinearRegression(), param_grid_search={'fit_intercept': [True, False]}, model_name='sklearn.linear_model.LinearRegression'),
 InitializedModelDetail(model_serial_number='module_1', model=RandomForestRegressor(min_samples_leaf=3), param_grid_search={'min_samples_leaf': [6]}, model_name='sklearn.ensemble.RandomForestRegressor')]

In [79]:
import numpy as np
import pandas as pd